In [ ]:
!pip install langchain-community


In [ ]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
pip install pinecone

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_2Y7bDH_4oYyB2hHTaMzvyQaYi1kZYL8tVaDMAGbdPWFJwvsZs7vWrL5stJRCmkTEZCb9kR")

In [ ]:
index_name = "developer-quickstart-py"

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )

In [ ]:
import os
from dotenv import load_dotenv
import uuid
from openai import OpenAI
import pinecone
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# -------------------------
# 1. Load API keys
# -------------------------
load_dotenv()
openai_key = os.getenv("API_KEY_OPENAI")       # 🔑 Match with .env
pinecone_key = os.getenv("API_KEY_PINECONE")   # 🔑 Match with .env

if not openai_key or not pinecone_key:
    raise ValueError("❌ Missing API keys. Check your .env file.")

# -------------------------
# 2. Extract text from PDF
# -------------------------
pdf_path = r"E:\Plant_ML\AI\Roger S. Pressman_ Bruce R. Maxin - Software Engineering_ A Practitioner’s Approach-McGraw-Hill Edu.pdf"
reader = PdfReader(pdf_path)

full_text = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        full_text += text + "\n"

# -------------------------
# 3. Split text into chunks
# -------------------------
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    length_function=len
)
chunks = text_splitter.split_text(full_text)

# -------------------------
# 4. Initialize clients
# -------------------------
client = OpenAI(api_key=openai_key)

# ✅ Correct Pinecone init
pc = pinecone.Pinecone(api_key=pinecone_key)
index_name = "developer-quickstart-py"
index = pc.Index(index_name)

# -------------------------
# 5. Embed and store chunks
# -------------------------
for i, chunk in enumerate(chunks):
    embedding_response = client.embeddings.create(
        model="text-embedding-3-small",  # ✅ use OpenAI’s embedding model
        input=chunk
    )
    vector = embedding_response.data[0].embedding

    index.upsert([
        {
            "id": str(uuid.uuid4()),
            "values": vector,
            "metadata": {"chunk_text": chunk, "page": i}
        }
    ])

print(f"✅ Stored {len(chunks)} chunks from PDF into Pinecone")

# -------------------------
# 6. Query the index
# -------------------------
query_text = "Explain the Waterfall software process model"

query_embedding = client.embeddings.create(
    model="text-embedding-3-small",
    input=query_text
).data[0].embedding

results = index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True
)

print("\n🔎 Top Retrieved Chunks:")
for match in results["matches"]:
    print(f"Score: {match['score']:.4f}")
    print(f"Page: {match['metadata']['page']}")
    print(f"Text: {match['metadata']['chunk_text'][:300]}...\n")

# -------------------------
# 7. Generate final answer
# -------------------------
context = "\n\n".join([m["metadata"]["chunk_text"] for m in results["matches"]])

completion = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are an expert tutor in software engineering."},
        {"role": "user", "content": f"Based on the following context, answer the question:\n\n{context}\n\nQuestion: {query_text}"}
    ]a
)

print("\n📝 Final Answer:")
print(completion.choices[0].message.content)


In [ ]:
pip install python-dotenv pinecone-client pypdf langchain google-generativeai


In [ ]:
from pinecone import Pinecone

# Initialize Pinecone
pc = Pinecone(api_key="pcsk_2Y7bDH_4oYyB2hHTaMzvyQaYi1kZYL8tVaDMAGbdPWFJwvsZs7vWrL5stJRCmkTEZCb9kR")

index_name = "developer-quickstart-py"

# If index already exists with wrong dimension, delete it first
if pc.has_index(index_name):
    pc.delete_index(index_name)

# Create new index with correct embedding model dimension
pc.create_index_for_model(
    name=index_name,
    cloud="aws",
    region="us-east-1",
    embed={
        "model": "llama-text-embed-v2",   # this will auto-set dimension to 1024
        "field_map": {"text": "chunk_text"}
    }
)

# Connect to the index
index = pc.Index(index_name)

print(f"Index {index_name} is ready with correct dimensions.")


In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_2Y7bDH_4oYyB2hHTaMzvyQaYi1kZYL8tVaDMAGbdPWFJwvsZs7vWrL5stJRCmkTEZCb9kR")

# delete old index
pc.delete_index("developer-quickstart-py")

# create new index with 768 dimension
pc.create_index(
    name="developer-quickstart-py",
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)


In [ ]:
import pinecone
import os
from dotenv import load_dotenv

# Load Pinecone key
load_dotenv()
pinecone_key = os.getenv("API_KEY_PINECONE")

# Initialize Pinecone
pc = pinecone.Pinecone(api_key=pinecone_key)
index_name = "developer-quickstart-py"
index = pc.Index(index_name)

# Delete all vectors in the index
index.delete(delete_all=True)

print("✅ All vectors cleared from Pinecone index.")


In [ ]:
import os
from dotenv import load_dotenv
import uuid
import pinecone
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import google.generativeai as genai

# -------------------------
# 1. Load API keys
# -------------------------
load_dotenv()
gemini_key = os.getenv("API_KEY_GEMINI")       # 🔑 Gemini key in .env
pinecone_key = os.getenv("API_KEY_PINECONE")   # 🔑 Pinecone key in .env

if not gemini_key or not pinecone_key:
    raise ValueError("❌ Missing API keys. Check your .env file.")

# Configure Gemini
genai.configure(api_key=gemini_key)

# -------------------------
# 2. Extract text from PDF
# -------------------------
pdf_path = r"E:\Plant_ML\AI\Roger S. Pressman_ Bruce R. Maxin - Software Engineering_ A Practitioner’s Approach-McGraw-Hill Edu.pdf"
reader = PdfReader(pdf_path)

full_text = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        full_text += text + "\n"

# -------------------------
# 3. Split text into chunks
# -------------------------
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    length_function=len
)
chunks = text_splitter.split_text(full_text)

# -------------------------
# 4. Initialize Pinecone
# -------------------------
pc = pinecone.Pinecone(api_key=pinecone_key)
index_name = "developer-quickstart-py"
index = pc.Index(index_name)

# -------------------------
# 5. Embed and store chunks
# -------------------------
for i, chunk in enumerate(chunks):
    embedding_response = genai.embed_content(
        model="models/embedding-001",  # ✅ Gemini embedding model
        content=chunk
    )
    vector = embedding_response["embedding"]

    index.upsert([
        {
            "id": str(uuid.uuid4()),
            "values": vector,
            "metadata": {"chunk_text": chunk, "page": i}
        }
    ])

print(f"✅ Stored {len(chunks)} chunks from PDF into Pinecone")

# -------------------------
# 6. Query the index
# -------------------------
query_text = "Explain the Waterfall software process model"

query_embedding = genai.embed_content(
    model="models/embedding-001",
    content=query_text
)["embedding"]

results = index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True
)

print("\n🔎 Top Retrieved Chunks:")
for match in results["matches"]:
    print(f"Score: {match['score']:.4f}")
    print(f"Page: {match['metadata']['page']}")
    print(f"Text: {match['metadata']['chunk_text'][:300]}...\n")

# -------------------------
# 7. Generate final answer
# -------------------------
context = "\n\n".join([m["metadata"]["chunk_text"] for m in results["matches"]])

model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content(
    f"Based on the following context, answer the question:\n\n{context}\n\nQuestion: {query_text}"
)

print("\n📝 Final Answer:")
print(response.text)


In [ ]:
import os
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json

# -------------------------
# PDF to chunks
# -------------------------
pdf_path = r"E:\Plant_ML\AI\Roger S. Pressman_ Bruce R. Maxin - Software Engineering_ A Practitioner’s Approach-McGraw-Hill Edu.pdf"
output_folder = r"E:\Plant_ML\AI\pdf_chunks"   # folder to save chunks
os.makedirs(output_folder, exist_ok=True)

reader = PdfReader(pdf_path)
full_text = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        full_text += text + "\n"

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    length_function=len
)
chunks = text_splitter.split_text(full_text)

# Save chunks as JSON (one file) or individual text files
for i, chunk in enumerate(chunks):
    with open(os.path.join(output_folder, f"chunk_{i}.txt"), "w", encoding="utf-8") as f:
        f.write(chunk)

print(f"✅ Saved {len(chunks)} chunks into {output_folder}")


In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_2t6mSD_RayKmzruxJCHqLK3yYSMkGVU7tSitdiF2dvyob3vVYkTcpMVreLQRmsxerCkpRv")

In [ ]:
index_name = "acadmate"

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )

In [ ]:
# Create Pinecone index for Gemini embeddings
pc.create_index(
    name="acadmate-gemini",
    dimension=768,          # Gemini embedding dimension
    metric="cosine",
    spec={"serverless": {"cloud": "aws", "region": "us-east-1"}}
)


In [ ]:
import os
import uuid
from dotenv import load_dotenv
from pinecone import Pinecone
import google.generativeai as genai

# -------------------------
# 1. Load API keys
# -------------------------
load_dotenv()
gemini_key = os.getenv("API_KEY_GEMINI")
pinecone_key = os.getenv("API_KEY_PINECONE")

if not gemini_key or not pinecone_key:
    raise ValueError("❌ Missing API keys. Check your .env file.")

genai.configure(api_key=gemini_key)

# -------------------------
# 2. Init Pinecone
# -------------------------
pc = Pinecone(api_key=pinecone_key)
index_name = "acadmate-gemini"   # ✅ your new Gemini-compatible index
index = pc.Index(index_name)

# -------------------------
# 3. Load chunks from folder
# -------------------------
chunk_folder = r"E:\Plant_ML\AI\pdf_chunks"
files = sorted(os.listdir(chunk_folder))

batch_size = 50
to_upsert = []

for i, filename in enumerate(files):
    with open(os.path.join(chunk_folder, filename), "r", encoding="utf-8") as f:
        chunk = f.read()

    # Gemini embedding (768-dim)
    embedding_response = genai.embed_content(
        model="models/embedding-001",
        content=chunk
    )
    vector = embedding_response["embedding"]

    to_upsert.append({
        "id": str(uuid.uuid4()),
        "values": vector,
        "metadata": {"chunk_text": chunk, "chunk_file": filename}
    })

    # Batch upsert
    if len(to_upsert) >= batch_size:
        index.upsert(to_upsert)
        to_upsert = []

if to_upsert:
    index.upsert(to_upsert)

print(f"✅ Stored {len(files)} chunks into Pinecone index '{index_name}'")

# -------------------------
# 4. Query Pinecone
# -------------------------
query_text = "Explain the Waterfall software process model"

query_embedding = genai.embed_content(
    model="models/embedding-001",
    content=query_text
)["embedding"]

results = index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True
)

print("\n🔎 Top Retrieved Chunks:")
for match in results.matches:
    print(f"Score: {match.score:.4f}")
    print(f"File: {match.metadata['chunk_file']}")
    print(f"Text: {match.metadata['chunk_text'][:300]}...\n")

# -------------------------
# 5. Generate final answer
# -------------------------
context = "\n\n".join([m.metadata["chunk_text"] for m in results.matches])

model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content(
    f"Based on the following context, answer the question:\n\n{context}\n\nQuestion: {query_text}"
)

print("\n📝 Final Answer:")
print(response.text)


In [1]:
"""
RAG Chatbot for exam-style answers + automatic grading
Requirements:
  - .env set with API_KEY_GEMINI and API_KEY_PINECONE
  - Pinecone index: 'acadmate-gemini' (768-dim embeddings)
  - Python packages: google-generativeai, pinecone-client (or pinecone), python-dotenv, textwrap
"""

import os
import textwrap
from dotenv import load_dotenv
from pinecone import Pinecone
import google.generativeai as genai

# -------------------------
# Config / init
# -------------------------
load_dotenv()
GEMINI_KEY = os.getenv("API_KEY_GEMINI")
PINECONE_KEY = os.getenv("API_KEY_PINECONE")
INDEX_NAME = "acadmate-gemini"   # your Gemini-compatible index

if not GEMINI_KEY or not PINECONE_KEY:
    raise RuntimeError("Missing API keys in .env (API_KEY_GEMINI, API_KEY_PINECONE)")

genai.configure(api_key=GEMINI_KEY)
pc = Pinecone(api_key=PINECONE_KEY)
index = pc.Index(INDEX_NAME)

# -------------------------
# Helpers
# -------------------------
def parse_rubric_input(rubric_str):
    """
    Parse simple rubric text like:
      "introduction:2, theory:5, example:3"
    returns list of (criterion, marks)
    """
    items = []
    for part in rubric_str.split(","):
        part = part.strip()
        if not part:
            continue
        if ":" in part:
            crit, m = part.split(":", 1)
            try:
                items.append((crit.strip(), int(m.strip())))
            except:
                items.append((crit.strip(), float(m.strip())))
        else:
            # fallback: treat as unnamed criterion (whole marks)
            items.append((part, 0))
    return items

def embed_query(text):
    resp = genai.embed_content(model="models/embedding-001", content=text)
    return resp["embedding"]

def retrieve_context(query_text, top_k=6):
    emb = embed_query(query_text)
    results = index.query(vector=emb, top_k=top_k, include_metadata=True)
    # Return list of chunk texts with scores
    retrieved = []
    for match in results.matches:
        chunk = match.metadata.get("chunk_text", "")
        retrieved.append({"score": match.score, "text": chunk, "file": match.metadata.get("chunk_file")})
    return retrieved

def build_system_prompt(rubric_items, total_marks, guidance=None):
    """
    Build a system-level instruction for the model describing the evaluation scheme.
    """
    rubric_lines = []
    for crit, m in rubric_items:
        rubric_lines.append(f"- {crit} : {m} marks")
    rubric_text = "\n".join(rubric_lines)
    guidance_text = guidance.strip() if guidance else "Answer precisely and concisely. Use clear headings and allocate content according to marks."
    sys = (
        "You are an expert university-level examiner and tutor. "
        "Given a question and a marks-scheme (rubric), produce a model answer aimed at a student "
        "that exactly follows the rubric: include headings for each rubric criterion, allocate effort proportional "
        "to the marks, and use numbered/bullet points if helpful. "
        "After the model answer, provide a concise marking sheet that assigns marks per criterion and a short justification "
        "for each awarded mark (self-grade). Do NOT invent facts beyond the provided context; prefer content from the context. "
        f"Total marks: {total_marks}\n\nRubric:\n{rubric_text}\n\nGuidance for style: {guidance_text}\n\n"
        "Output format:\n"
        "### Model Answer (student-facing)\n"
        "<content>\n\n"
        "### Marking (teacher-facing)\n"
        "- criterion: awarded_marks / max_marks — justification (1-2 sentences)\n"
        "Total: X / TOTAL\n"
    )
    return sys

def compose_prompt(context_chunks, question, system_instructions):
    """
    Compose final prompt. We give the context (retrieved chunks), the question, and the instructions.
    """
    # Take top N chunks' text concatenated, but keep them short - mark origins
    ctx_parts = []
    for i, c in enumerate(context_chunks):
        ctx_parts.append(f"[Chunk {i+1} | score={c['score']:.4f}]\n{c['text']}\n")
    context = "\n\n".join(ctx_parts)
    prompt = (
        f"{system_instructions}\n\n"
        f"CONTEXT (from the textbook / notes):\n{context}\n\n"
        f"QUESTION: {question}\n\n"
        "Generate the Model Answer and the Marking as specified above.\n"
    )
    return prompt

def call_gemini_generate(prompt, model_name="gemini-1.5-flash"):
    gm = genai.GenerativeModel(model_name)
    # Use generate_content with text input
    response = gm.generate_content(prompt)
    # The SDK returns an object. We try to extract text safely:
    try:
        return response.text
    except AttributeError:
        # fallback - examine fields (some SDK versions return different fields)
        return getattr(response, "output_text", str(response))

def pretty_print_long(text):
    print("\n" + "-"*80 + "\n")
    for line in textwrap.wrap(text, width=100):
        print(line)
    print("\n" + "-"*80 + "\n")

# -------------------------
# Interactive flow
# -------------------------
def run_chatbot():
    print("RAG Exam Chatbot — generate answers tailored to a rubric + self-grade")
    print("Type 'exit' to quit.\n")

    while True:
        # Get question (direct entry or file path)
        q_input = input("Enter question text (or path to .txt file): ").strip()
        if q_input.lower() in ("exit", "quit", "q"):
            break

        if os.path.isfile(q_input):
            with open(q_input, "r", encoding="utf-8") as fh:
                question = fh.read().strip()
        else:
            question = q_input

        # Marks and rubric
        total_marks_input = input("Total marks (e.g., 10): ").strip()
        try:
            total_marks = int(total_marks_input)
        except:
            total_marks = int(float(total_marks_input))

        rubric_input = input("Rubric (format: criterion1:marks, criterion2:marks). Example: 'Intro:2, Theory:6, Example:2'\nEnter rubric: ").strip()
        rubric_items = parse_rubric_input(rubric_input)
        # If rubric sums to 0 or sums != total_marks, normalize or warn
        sum_marks = sum([m for _, m in rubric_items])
        if sum_marks == 0:
            # default single criterion equal to total
            rubric_items = [("Answer", total_marks)]
            sum_marks = total_marks
        if sum_marks != total_marks:
            print(f"[!] Rubric total {sum_marks} != Total marks {total_marks}. I'll keep rubric marks as provided and set Total={sum_marks}.")
            total_marks = sum_marks

        # Optional guidance
        guidance = input("Any special guidance for the answer (tone/length/keywords)? (press Enter to skip): ").strip()

        # Retrieve context
        print("\nRetrieving context from Pinecone...")
        retrieved = retrieve_context(question, top_k=6)
        print(f"Retrieved {len(retrieved)} chunks. Top scores: {[round(r['score'], 4) for r in retrieved[:5]]}")

        # Build system instructions & prompt
        sys_inst = build_system_prompt(rubric_items, total_marks, guidance=guidance)
        prompt = compose_prompt(retrieved, question, sys_inst)

        print("\nGenerating answer with Gemini (may take a few seconds)...")
        result_text = call_gemini_generate(prompt)

        # Print result
        print("\n\n=== MODEL OUTPUT ===")
        pretty_print_long(result_text)

        # Optionally save to file
        save_opt = input("Save this output to file? (y/n): ").strip().lower()
        if save_opt == "y":
            safe_fname = "rag_answer_output.txt"
            with open(safe_fname, "w", encoding="utf-8") as fh:
                fh.write(result_text)
            print(f"Saved to {safe_fname}")

if __name__ == "__main__":
    run_chatbot()


KeyboardInterrupt: 

In [2]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
import pinecone

# Load .env file
load_dotenv()

gemini_key = os.getenv("API_KEY_GEMINI")
pinecone_key = os.getenv("API_KEY_PINECONE")

# Configure Gemini
genai.configure(api_key=gemini_key)

# Configure Pinecone
pc = pinecone.Pinecone(api_key=pinecone_key)
index = pc.Index("acadmate-gemini")   # change to your index name


In [4]:
def generate_answer(question, marks):
    # Step 1: Embed question using Gemini
    embed_model = "models/embedding-001"
    query_embedding = genai.embed_content(
        model=embed_model,
        content=question,
        task_type="retrieval_query"
    )["embedding"]

    # Step 2: Query Pinecone
    result = index.query(vector=query_embedding, top_k=3, include_metadata=True)
    retrieved_text = " ".join([match.metadata["text"] for match in result.matches])

    # Step 3: Build system rules
    system_rules = f"""
    You are an examiner chatbot.
    Rules:
    - Use retrieved textbook content only.
    - Write answers in exam-style structured format:
      1. Definition (short paragraph)
      2. Explanation (points)
      3. Functions / Advantages / Working (points if relevant)
      4. Example (if available)
      5. Diagram Reference (if applicable, write: 'Refer textbook for diagram')
      6. Conclusion (1-2 lines)
    - Length Control:
      - 2 marks → 3-4 lines
      - 5 marks → 10-15 lines
      - 10 marks → 20-25 lines
    - All explanation except definition must be point-wise.
    - Maintain textbook style language.
    """

    user_prompt = f"""
    Question: {question}
    Marks: {marks}
    Retrieved Textbook Content: {retrieved_text}
    """

    # Step 4: Call Gemini
    model = genai.GenerativeModel("gemini-pro")
    response = model.generate_content(system_rules + "\n" + user_prompt)

    return response.text


In [8]:
print(generate_answer("Explain kernel in OS", 5))


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerDayPerUserPerProjectPerModel-FreeTier"
}
violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerMinutePerUserPerProjectPerModel-FreeTier"
}
violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerMinutePerProjectPerModel-FreeTier"
}
violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerDayPerProjectPerModel-FreeTier"
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rat

In [6]:
import google.generativeai as genai
genai.configure(api_key=gemini_key)

print(genai.get_model("models/embedding-001"))


Model(name='models/embedding-001',
      base_model_id='',
      version='001',
      display_name='Embedding 001',
      description='Obtain a distributed representation of a text.',
      input_token_limit=2048,
      output_token_limit=1,
      supported_generation_methods=['embedContent'],
      temperature=None,
      max_temperature=None,
      top_p=None,
      top_k=None)


In [9]:
from dotenv import load_dotenv
import os
import google.generativeai as genai

load_dotenv()
gemini_key = os.getenv("API_KEY_GEMINI")
pinecone_key = os.getenv("API_KEY_PINECONE")

genai.configure(api_key=gemini_key)


In [10]:
print(genai.get_model("models/embedding-001"))


Model(name='models/embedding-001',
      base_model_id='',
      version='001',
      display_name='Embedding 001',
      description='Obtain a distributed representation of a text.',
      input_token_limit=2048,
      output_token_limit=1,
      supported_generation_methods=['embedContent'],
      temperature=None,
      max_temperature=None,
      top_p=None,
      top_k=None)


In [11]:
print(generate_answer("Explain kernel in OS", 5))


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerDayPerUserPerProjectPerModel-FreeTier"
}
violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerMinutePerUserPerProjectPerModel-FreeTier"
}
violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerMinutePerProjectPerModel-FreeTier"
}
violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerDayPerProjectPerModel-FreeTier"
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rat

In [12]:
def generate_answer(question, marks):
    # TEMP: Instead of embedding with Gemini, use a simple string filter for Pinecone
    result = index.query(vector=[0.0]*768,  # dummy vector if needed
                         filter={"text": {"$contains": question}},
                         top_k=3,
                         include_metadata=True)
    
    context = " ".join([m["metadata"]["text"] for m in result["matches"]])

    model = genai.GenerativeModel("gemini-pro")
    prompt = f"""
    Question: {question}
    Marks: {marks}

    Context (from textbook): {context}

    Rules:
    - If 5 marks → write 10-15 lines
    - If 10 marks → write 20-25 lines
    - Write point-wise except for definitions
    - If diagram is needed → say 'Refer textbook for diagram'
    """

    response = model.generate_content(prompt)
    return response.text


In [13]:
def generate_answer(question, marks):
    # Step 1: Retrieve textbook chunks (dummy vector for testing)
    result = index.query(
        vector=[0.0]*768,  # dummy vector, Option A
        top_k=3,
        include_metadata=True
    )
    
    # Combine retrieved chunks
    context = " ".join([match.metadata["text"] for match in result.matches])

    # Step 2: Build prompt with rules
    prompt = f"""
Question: {question}
Marks: {marks}

Context (from textbook): {context}

Answer Rules:
- 5 marks → 10-15 lines
- 10 marks → 20-25 lines
- 2 marks → 3-4 lines
- Write point-wise (except definitions)
- If diagram is needed → say 'Refer textbook for diagram'
- Use textbook style language
- Structured format:
  1. Definition
  2. Explanation (points)
  3. Functions / Advantages / Working (points if relevant)
  4. Example (if available)
  5. Diagram Reference
  6. Conclusion
"""

    # Step 3: Generate answer using Gemini-Pro
    model = genai.GenerativeModel("gemini-1.5")
    response = model.generate_content(prompt)
    
    return response.text


In [14]:
question = "Explain kernel in OS"
marks = 5  # change to 2, 5, 10 as needed

answer = generate_answer(question, marks)
print(answer)


KeyError: 'text'